# Introduction to encoding textual data

In [2]:
import pandas as pd
import numpy as np
import pymongo
from nltk.tokenize import sent_tokenize
from datasets import load_dataset

# Remember to start Mongo with sudo systemctl start mongod

In [ ]:
# Load dataset into DB

db = pymongo.MongoClient()['climate']
ds = load_dataset("tdiggelm/climate_fever")

evidences = []
for record in ds['test']:
    for evidence in record['evidences']:
        evidence['claim_id'] = record['claim_id']
        evidence['claim'] = record['claim']
        evidence['claim_label'] = record['claim_id']
        evidences.append(evidence)

db['evidences'].insert_many(evidences)
